### Import all packages

In [1]:
import sys
sys.path.append("..") 

In [2]:
import cv2
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import torch
from pathlib import Path
import os
import torch.cuda
import albumentations as albu
import segmentation_models_pytorch as smp
from  segmentation_models_pytorch.utils.base import Metric
from segmentation_models_pytorch.base.modules import Activation
from collections import defaultdict
import nibabel as nib
from tqdm import tqdm
from Utils.dataset_utils import *

In [3]:
from torchsummary import summary
from torch.utils.data import DataLoader
from torch.utils.data import Dataset as BaseDataset

In [32]:
VOLUMES_BASE_PATH=Path(r'G:\Projects and Work\Mouse Heart Segmentation\Mice_CT_Dataset')
LABELS_TO_KEEP=[1,600,420,550,205]
SUBJECTS=os.listdir(VOLUMES_BASE_PATH/'volumes')
SUBJECTS=list(filter(lambda x: x.find("LAD") != -1,SUBJECTS))
print(SUBJECTS)

['LAD-#11-MicroCT-050321 - Cardiac CT_133143-diastole', 'LAD-#11-MicroCT-050321 - Cardiac CT_133143-systole', 'LAD-#12-MicroCT-050321 - Cardiac CT_133905-diastole', 'LAD-#12-MicroCT-050321 - Cardiac CT_133905-systole', 'LAD-#6-MicroCT-050321 - Cardiac CT_125739-diastole', 'LAD-#6-MicroCT-050321 - Cardiac CT_125739-systole', 'LAD-Uma#1-MicroCT-030519 - Cardiac CT_133621-diastole', 'LAD-Uma#1-MicroCT-030519 - Cardiac CT_133621-systole', 'LAD-Uma#5-MicroCT-030519 - Cardiac CT_134513-diastole', 'LAD-Uma#5-MicroCT-030519 - Cardiac CT_134513-systole']


In [20]:
os.listdir(VOLUMES_BASE_PATH)

['Axials', 'labels', 'volumes']

In [21]:
WIDTH=512
HEIGHT=512

In [22]:
def convert_labels(label_volume):
    new_labels=np.zeros(label_volume.shape)
    for lbl in np.unique(label_volume):
        if lbl in LABELS_TO_KEEP:
            new_labels[np.where(label_volume==lbl)]=1
    return new_labels

def to_tensor(x, **kwargs): 
    x= torch.from_numpy(x.transpose(2, 0, 1).astype('float32'))
    return x

def create_folder(path):
    if os.path.exists(path)==False:
        os.mkdir(path)
    

In [23]:
def predict_img(model,img):
    img=preprocessing_fn(img)
    img=to_tensor(img)
    img = img.to(device=DEVICE)
    img=torch.unsqueeze(img,dim=0)
    pred_mask=torch.squeeze(model(img))
    pred_mask=pred_mask.detach().cpu().numpy()
    return pred_mask
def prepare_slice(img):
    img=normalize(img)
    img = img*255
    img =img.astype('uint8')
    img = cv2.cvtColor(img,cv2.COLOR_GRAY2BGR)
    return img
def predict_volume(model,volume,all_axials=False):
    pred_vol=np.zeros(volume.shape)
    out_Z=np.zeros(volume.shape)
    if all_axials:
        out_X=np.zeros(volume.shape)
        out_Y=np.zeros(volume.shape)
    X,Y,Z=volume.shape
    for i in tqdm(range(Z)):
        img = prepare_slice(volume[:,:,i])
        pred_mask=predict_img(model,img)
        out_Z[:,:,i]=pred_mask
        
    if all_axials:
        for i in tqdm(range(Y)):
            img = prepare_slice(volume[:,i,:])
            pred_mask=predict_img(model,img)
            out_Y[:,i,:]=pred_mask
        for i in tqdm(range(X)):
            img = prepare_slice(volume[i,:,:])
            pred_mask=predict_img(model,img)
            out_X[i,:,:]=pred_mask
        pred_vol=(out_X+out_Y+out_Z)/3.0
        pred_vol[pred_vol>0.5]=1
        pred_vol[pred_vol<0.5]=0
        return pred_vol
    else:
        return out_Z

In [24]:
# Model to choose
ENCODER = 'efficientnet-b4'
ENCODER_WEIGHTS = 'imagenet'
DEVICE = 'cuda'
MODEL_NAME='Unet'
BEST_WEIGHTS=fr'G:\Projects and Work\Mouse Heart Segmentation\Trained Weights - 10 Mice Volumes\Unet_{ENCODER}\best.pt'
OUTPUT_PATH=Path(fr'G:\Projects and Work\Mouse Heart Segmentation\Trained Weights - 10 Mice Volumes\Unet_{ENCODER}\Outputs New')
create_folder(OUTPUT_PATH)

In [25]:
# Load model
preprocessing_fn = smp.encoders.get_preprocessing_fn(ENCODER, ENCODER_WEIGHTS)
best_model = torch.load(BEST_WEIGHTS)

In [26]:
# function to save numpy volume array to .nii
def save_mask_nii(volume_arr,affine,path):
    ni_img = nib.Nifti1Image(volume_arr,affine)
    nib.save(ni_img, path)

In [33]:
SUBJECTS

['LAD-#11-MicroCT-050321 - Cardiac CT_133143-diastole',
 'LAD-#11-MicroCT-050321 - Cardiac CT_133143-systole',
 'LAD-#12-MicroCT-050321 - Cardiac CT_133905-diastole',
 'LAD-#12-MicroCT-050321 - Cardiac CT_133905-systole',
 'LAD-#6-MicroCT-050321 - Cardiac CT_125739-diastole',
 'LAD-#6-MicroCT-050321 - Cardiac CT_125739-systole',
 'LAD-Uma#1-MicroCT-030519 - Cardiac CT_133621-diastole',
 'LAD-Uma#1-MicroCT-030519 - Cardiac CT_133621-systole',
 'LAD-Uma#5-MicroCT-030519 - Cardiac CT_134513-diastole',
 'LAD-Uma#5-MicroCT-030519 - Cardiac CT_134513-systole']

In [35]:
# Run a model on all volumes

for i in range(len(SUBJECTS)):
    print(SUBJECTS[i])
    subject_name = SUBJECTS[i]
    subject_path = os.path.join(VOLUMES_BASE_PATH,'volumes',subject_name,f'{subject_name}_volume.nii') # volume path
    if os.path.exists(subject_path) is False: #if .nii file of volume ct not present then skip
        continue
    subject_label_path = os.path.join(VOLUMES_BASE_PATH,'labels',subject_name ,f'{subject_name}_label.nii') #volume label
    if os.path.exists(subject_label_path)==False: # checking if extension is .nii or .nii.gz 
        subject_label_path = os.path.join(VOLUMES_BASE_PATH,'labels',subject_name,f'{subject_name}_label.nii.gz')   
    if os.path.exists(os.path.join(OUTPUT_PATH,subject_name)):
        continue
    create_folder(os.path.join(OUTPUT_PATH,subject_name)) # create output folder
    volume, volume_gt_mask,affine = load_case(subject_path,subject_label_path) # Load volume with its affine and mask
    volume_pred_mask=predict_volume(best_model,volume,True)   # Predict volume
    volume_pred_mask=np.round(volume_pred_mask)
    save_mask_nii(volume_pred_mask,affine,os.path.join(OUTPUT_PATH,subject_name,'prediction.nii.gz'))    #save predicted volume
    if volume_gt_mask is not None:     # save grouth truth .nii file also for convinience
        save_mask_nii(volume_gt_mask,affine,os.path.join(OUTPUT_PATH,subject_name,'ground_truth.nii.gz'))

LAD-#11-MicroCT-050321 - Cardiac CT_133143-diastole
G:\Projects and Work\Mouse Heart Segmentation\Mice_CT_Dataset\labels\LAD-#11-MicroCT-050321 - Cardiac CT_133143-diastole\LAD-#11-MicroCT-050321 - Cardiac CT_133143-diastole_label.nii.gz


100%|█████████████████████████████████████████████████████████████████████████████████████████████████████████████| 512/512 [00:37<00:00, 13.66it/s]


LAD-#11-MicroCT-050321 - Cardiac CT_133143-systole
G:\Projects and Work\Mouse Heart Segmentation\Mice_CT_Dataset\labels\LAD-#11-MicroCT-050321 - Cardiac CT_133143-systole\LAD-#11-MicroCT-050321 - Cardiac CT_133143-systole_label.nii.gz


100%|█████████████████████████████████████████████████████████████████████████████████████████████████████████████| 512/512 [00:35<00:00, 14.40it/s]


LAD-#12-MicroCT-050321 - Cardiac CT_133905-diastole
G:\Projects and Work\Mouse Heart Segmentation\Mice_CT_Dataset\labels\LAD-#12-MicroCT-050321 - Cardiac CT_133905-diastole\LAD-#12-MicroCT-050321 - Cardiac CT_133905-diastole_label.nii.gz


...

100%|█████████████████████████████████████████████████████████████████████████████████████████████████████████████| 512/512 [00:35<00:00, 14.43it/s]


LAD-Uma#5-MicroCT-030519 - Cardiac CT_134513-diastole
G:\Projects and Work\Mouse Heart Segmentation\Mice_CT_Dataset\labels\LAD-Uma#5-MicroCT-030519 - Cardiac CT_134513-diastole\LAD-Uma#5-MicroCT-030519 - Cardiac CT_134513-diastole_label.nii.gz


100%|█████████████████████████████████████████████████████████████████████████████████████████████████████████████| 512/512 [00:36<00:00, 14.15it/s]


LAD-Uma#5-MicroCT-030519 - Cardiac CT_134513-systole
G:\Projects and Work\Mouse Heart Segmentation\Mice_CT_Dataset\labels\LAD-Uma#5-MicroCT-030519 - Cardiac CT_134513-systole\LAD-Uma#5-MicroCT-030519 - Cardiac CT_134513-systole_label.nii.gz


100%|█████████████████████████████████████████████████████████████████████████████████████████████████████████████| 512/512 [00:35<00:00, 14.42it/s]


In [13]:
def visualize(**images):
    """PLot images in one row."""
    n = len(images)
    plt.figure(figsize=(16, 5))

    for i, (name, image) in enumerate(images.items()):
        plt.subplot(1, n, i + 1)
        plt.xticks([])
        plt.yticks([])
        plt.title(' '.join(name.split('_')).title())
        plt.imshow(image,cmap="gray")
    plt.show()

for i in range(10):
    n = np.random.choice(volume_pred_mask.shape[2])
    visualize(image=volume[:,:,n],gt_mask= volume_gt_mask[:,:,n],pred_mask=volume_pred_mask[:,:,n])


TypeError: 'NoneType' object is not subscriptable